In [1]:
from api_keys import weather_api_key
import pandas as pd
from sqlalchemy import create_engine
import requests
import datetime

# Extraction

In [2]:
csv_file = "California_Fire_Incidents.csv"
cal_Fire_Inci_df = pd.read_csv(csv_file)
cal_Fire_Inci_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [3]:
csv_file = "County_info.csv"
county_info = pd.read_csv(csv_file)
county_info

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,1996-01-31,...,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31
0,3101,0,Los Angeles County,County,CA,CA,Los Angeles-Long Beach-Anaheim,6,37,175538.0,...,631450.0,633373.0,636620.0,639983.0,645447.0,650877.0,655589.0,657455.0,658836.0,664094.0
1,139,1,Cook County,County,IL,IL,Chicago-Naperville-Elgin,17,31,151180.0,...,248414.0,248122.0,247915.0,247925.0,248136.0,248548.0,248838.0,249194.0,249714.0,250814.0
2,1090,2,Harris County,County,TX,TX,Houston-The Woodlands-Sugar Land,48,201,100916.0,...,200198.0,200692.0,201230.0,201360.0,201790.0,202689.0,203762.0,204598.0,205096.0,205702.0
3,2402,3,Maricopa County,County,AZ,AZ,Phoenix-Mesa-Scottsdale,4,13,117969.0,...,292676.0,294735.0,297121.0,299473.0,301627.0,304151.0,307591.0,310595.0,313263.0,315783.0
4,2841,4,San Diego County,County,CA,CA,San Diego-Carlsbad,6,73,184765.0,...,612972.0,615771.0,619048.0,623541.0,627005.0,630889.0,633214.0,636054.0,637310.0,639238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2852,425,3101,Camas County,County,ID,ID,Hailey,16,25,NaN,...,120391.0,121292.0,122184.0,123119.0,123611.0,123799.0,124029.0,124508.0,125039.0,125683.0
2853,1648,3102,Daggett County,County,UT,UT,NaN,49,9,NaN,...,209722.0,210257.0,210202.0,210193.0,210127.0,210629.0,210902.0,211365.0,211801.0,212864.0
2854,2345,3112,Hinsdale County,County,CO,CO,NaN,8,53,NaN,...,330363.0,331253.0,331744.0,332152.0,332593.0,333098.0,332018.0,331755.0,331616.0,332733.0
2855,1933,3122,Mineral County,County,CO,CO,NaN,8,79,NaN,...,283733.0,283414.0,282863.0,282891.0,283029.0,283264.0,283064.0,283001.0,282851.0,282782.0


In [4]:
print(cal_Fire_Inci_df.columns)
print(county_info.columns)

Index(['AcresBurned', 'Active', 'AdminUnit', 'AirTankers', 'ArchiveYear',
       'CalFireIncident', 'CanonicalUrl', 'ConditionStatement',
       'ControlStatement', 'Counties', 'CountyIds', 'CrewsInvolved', 'Dozers',
       'Engines', 'Extinguished', 'Fatalities', 'Featured', 'Final',
       'FuelType', 'Helicopters', 'Injuries', 'Latitude', 'Location',
       'Longitude', 'MajorIncident', 'Name', 'PercentContained',
       'PersonnelInvolved', 'Public', 'SearchDescription', 'SearchKeywords',
       'Started', 'Status', 'StructuresDamaged', 'StructuresDestroyed',
       'StructuresEvacuated', 'StructuresThreatened', 'UniqueId', 'Updated',
       'WaterTenders'],
      dtype='object')
Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'StateCodeFIPS', 'MunicipalCodeFIPS', '1996-01-31',
       ...
       '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
       '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30', '2020-0

In [5]:
# cal_Fire_Inci_df['Updated'] = pd.to_datetime(cal_Fire_Inci_df['Updated'])
# cal_Fire_Inci_df['Started'] = pd.to_datetime(cal_Fire_Inci_df['Started'])

# Transformation

In [6]:
CA_countys = county_info.loc[county_info['State'] == 'CA']
CA_countys['RegionName'] = CA_countys['RegionName'].str.extract(r'([\w ]*) County',expand=True)

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Drop unneeded data columns

In [7]:
first_date = "2012-12-31"
wrong_columns = CA_countys.loc[:,'Metro':first_date]
CA_countys = CA_countys.drop(columns = wrong_columns.columns)
end_date = "2020-01-31"
wrong_columns = CA_countys.loc[:,end_date:]
CA_countys = CA_countys.drop(columns = wrong_columns.columns)
CA_countys = CA_countys.drop(columns = ['SizeRank','RegionType','StateName'])

## Split normalize dataframe

In [8]:
countys = CA_countys['RegionName']
countys = countys.append(cal_Fire_Inci_df['Counties']).reset_index()
countys = countys.rename(columns={0:'RegionName'})
countys = countys.drop_duplicates('RegionName')
# countys.to_csv('Countys.csv')
countys

,index,RegionName
0,0,Los Angeles
1,4,San Diego
2,5,Orange
3,10,Riverside
4,11,San Bernardino
...,...,...
56,2817,Sierra
57,2851,Alpine
1479,1421,Mexico
1481,1423,State of Oregon


In [9]:
columns = CA_countys.loc[:,"2013-01-31":"2019-12-31"].columns
county_CA = pd.melt(CA_countys, id_vars=['RegionID', 'RegionName'], value_vars = columns)
county_CA

,RegionID,RegionName,variable,value
0,3101,Los Angeles,2013-01-31,390772.0
1,2841,San Diego,2013-01-31,395630.0
2,1286,Orange,2013-01-31,512190.0
3,2832,Riverside,2013-01-31,241946.0
4,3250,San Bernardino,2013-01-31,195566.0
...,...,...,...,...
4867,253,Mono,2019-12-31,460224.0
4868,2056,Trinity,2019-12-31,280572.0
4869,647,Modoc,2019-12-31,130233.0
4870,1391,Sierra,2019-12-31,239567.0


In [10]:
fires_db = cal_Fire_Inci_df[['UniqueId', 'Name']]
fires_db = fires_db.drop_duplicates('UniqueId')
Cal_Fire_db = cal_Fire_Inci_df[['UniqueId','Counties', 'ArchiveYear','AcresBurned',
                               'StructuresDamaged', 'StructuresDestroyed', 'Updated',
                               'Latitude', 'Longitude', 'Started']]
Cal_Fire_db = Cal_Fire_db.rename(columns = {'UniqueId':'fireid', 'ArchiveYear':'yearstarted',
        'AcresBurned':'acresburned', 'Counties':'countyid',
        'Updated':'extinguished','StructuresDamaged':'structuresdamaged',
        'StructuresDestroyed':'structuresdestroyed', 'Latitude':'latitude',
        'Longitude':'longitude', 'Started':'started'})
county_CA = county_CA.rename(columns = {'RegionID':'saleid', 'variable':'date', 'RegionName':'countyid',
       'value':'medianprice'})
countys = countys.rename(columns ={'index': 'countyid', 'RegionName': 'countyname'})
fires_db = fires_db.rename(columns ={'UniqueId': 'fireid', 'Name': 'name'})

In [11]:
Cal_Fire_db['countyid'] = Cal_Fire_db['countyid'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [12]:
county_CA['saleid'] = county_CA['medianprice'].apply(lambda s : abs(hash(s)) % (10 ** 8))
county_CA['saleid'] = county_CA['saleid']+county_CA['date'].apply(lambda s : abs(hash(s)) % (10 ** 8))
county_CA['countyid'] = county_CA['countyid'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [13]:
countys['countyid'] = countys['countyname'].apply(lambda s : abs(hash(s)) % (10 ** 8))

In [14]:
Cal_Fire_db = Cal_Fire_db.fillna(0)
county_CA = county_CA.fillna(0)
# Cal_Fire_db = Cal_Fire_db.set_index('fireid')
# fires_db = fires_db.set_index('fireid')
# county_CA = county_CA.set_index('countyid')
# countys = countys.set_index('countyid')

In [15]:
rds_connection_string = "postgres:Tgnome75@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [17]:
fires_db.to_sql(name='fire', con=engine, if_exists='append', index=False)
countys.to_sql(name='county', con=engine, if_exists='append', index=False)
county_CA.to_sql(name='salehistory', con=engine, if_exists='append', index=False)
Cal_Fire_db.to_sql(name='fires', con=engine, if_exists='append', index=False)

In [18]:
County = pd.read_sql_query('select * from county', con=engine)
SaleHistory = pd.read_sql_query('select * from salehistory', con=engine)
Fires = pd.read_sql_query('select * from fires', con=engine)
Fire = pd.read_sql_query('select * from fire', con=engine)

In [26]:
countyPrices = pd.merge(SaleHistory, County, on = 'countyid', how='right', validate = 'many_to_one')
countyPrices.head(15)

,saleid,countyid,date,medianprice,countyname
0,5605431.0,47613549,2013-01-31,390772.0,Los Angeles
1,10952310.0,47613549,2013-02-28,398166.0,Los Angeles
2,37392594.0,47613549,2013-03-31,405575.0,Los Angeles
3,33896162.0,47613549,2013-04-30,414911.0,Los Angeles
4,65771413.0,47613549,2013-05-31,422488.0,Los Angeles
5,24031345.0,47613549,2013-06-30,430180.0,Los Angeles
6,2031419.0,47613549,2013-07-31,437808.0,Los Angeles
7,50197793.0,47613549,2013-08-31,447415.0,Los Angeles
8,1985456.0,47613549,2013-09-30,455616.0,Los Angeles
9,84672061.0,47613549,2013-10-31,461311.0,Los Angeles


In [27]:
countyFires = pd.merge(Fires, County, on ='countyid', how = 'inner',validate = 'many_to_one')
countyFires = countyFires.merge(Fire, on = 'fireid', how = 'inner', validate = 'many_to_one')
countyFires.head(15)

,fireid,countyid,yearstarted,acresburned,structuresdamaged,structuresdestroyed,extinguished,latitude,longitude,started,countyname,name
0,5fb18d4d-213f-4d83-a179-daaf11939e78,23573616,2013,257314,0,0,2013-09-06,38,-120,2013-08-17,Tuolumne,Rim Fire
1,1e8441c2-f426-40b4-9f4f-3de9f7e232b9,23573616,2013,1070,0,0,2013-08-14,38,-120,2013-08-05,Tuolumne,Power Fire
2,98bfbb7c-8446-4e2e-a268-7e961353723c,23573616,2013,100,0,0,2013-06-02,38,-120,2013-06-02,Tuolumne,Murphy Fire
3,1a0bbf7e-7f9b-4359-9c91-85dec2e2869a,23573616,2014,487,0,0,2014-06-05,38,-120,2014-06-04,Tuolumne,59 Fire
4,b5d32131-cf00-4e34-9e08-b74d68a3174f,23573616,2014,269,0,0,2014-06-06,38,-120,2014-06-05,Tuolumne,Hetch Hetchy Fire
5,726710bb-32d4-46bf-9729-7f3396defcf0,23573616,2014,70,0,0,2014-09-05,38,-120,2014-09-04,Tuolumne,Jackson Fire
6,55a1ec97-90fb-48df-8414-203b91adcd76,23573616,2015,204,0,0,2015-08-04,38,-120,2015-07-29,Tuolumne,Big Creek Fire
7,72c7a5c9-82c2-44c4-8bf2-241fb36e1ee7,23573616,2015,112,0,0,2015-08-16,38,-121,2015-08-15,Tuolumne,Tulloch Fire
8,40f44488-051f-4412-a577-c932f2ce751d,23573616,2015,108,0,3,2015-09-15,38,-120,2015-09-08,Tuolumne,Oak Fire
9,3a4871dc-2299-4934-8a31-b92780fe31a3,23573616,2016,1080,0,0,2016-10-04,38,-120,2016-09-26,Tuolumne,Marshes Fire


In [28]:
County.head(15)

,countyid,countyname
0,47613549,Los Angeles
1,97060049,San Diego
2,42644866,Orange
3,93881486,Riverside
4,35688692,San Bernardino
5,43466012,Santa Clara
6,77613075,Alameda
7,13746539,Sacramento
8,64377249,Contra Costa
9,31323504,Fresno


In [29]:
Fire.head(15)

,fireid,name
0,5fb18d4d-213f-4d83-a179-daaf11939e78,Rim Fire
1,bf37805e-1cc2-4208-9972-753e47874c87,Powerhouse Fire
2,a3149fec-4d48-427c-8b2c-59e8b79d59db,Mountain Fire
3,8213f5c7-34fa-403b-a4bc-da2ace6e6625,American Fire
4,46731fb8-3350-4920-bdf7-910ac0eb715c,Springs Fire
5,bee8c339-4f26-4b78-a5b4-a8a0ebdb8786,Aspen Fire
6,c400203b-a7fd-4bd8-803a-f3c74bc32a2b,Silver Fire
7,ba76c009-09c9-497c-b923-1ac8cf05c20d,Salmon River Complex
8,f3dcbca8-f8ed-46d0-8f6e-9c7fc2fb03e6,Corral Complex
9,956dbcf6-db40-4b61-b235-4ede14738f1e,Deer Fire


In [ ]:
# Summary of crews
# Summary by year
# Summary by counties
# Summary by 
# year = 2013
# for index,row in cal_Fire_Inci_df[['Latitude','Longitude', 'ArchiveYear','Counties']].iterrows():
#     if row[2] != year:
#         year += 1
#     try:
#         url = f'https://history.openweathermap.org/data/2.5/aggregated/year={year}lat={row[0]}&lon={row[1]}&appid={weather_api_key}&units=metric'
        
#         city_data = requests.get(url).json()
# #         print(f'Processing Record {record_counter} of Set {set_counter} | {row[3]} County')
#         print(city_data.jsonify())
# #         city_data_list.append([{row},city_data['coord']['lat'],city_data['coord']['lon'],
# #                                city_data['main']['temp_max'],city_data['main']['humidity'],
# #                                city_data['clouds']['all'], city_data['wind']['speed'],
# #                                city_data['sys']['country'],city_data['dt']])
#     except:
#         print('City not found. Skipping...')
#         print(city_data)
#     break;



# year = 2013
# for index,row in cal_Fire_Inci_df[['Latitude','Longitude', 'ArchiveYear','Counties']].iterrows():
#     if row[2] != year:
#         year += 1
#     try:
#         url = f'https://history.openweathermap.org/data/2.5/aggregated/year=?lat={lat}&lon={lon}&appid={weather_api_key}&units=metric'
# #         http://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start}&end={end}&appid={YOUR_API_KEY}

#         city_data = requests.get(url).json()
#         print(f'Processing Record {record_counter} of Set {set_counter} | {city}')
#         city_data_list.append([city,city_data['coord']['lat'],city_data['coord']['lon'],
#                                city_data['main']['temp_max'],city_data['main']['humidity'],
#                                city_data['clouds']['all'], city_data['wind']['speed'],
#                                city_data['sys']['country'],city_data['dt']])
#     except:
#         print('City not found. Skipping...')
#         print(city_data)